In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

# Test 1: Simple non-DB, non-Snorkel

In [2]:
xs         = range(100)
y_set_gold = set(range(1,101))

In [3]:
from snorkel.udf import UDF, UDFRunner, UDFRunnerMP
from time import sleep

class AddOne(UDF):
    def apply(self, x):
        sleep(1)
        yield x + 1

### First, sanity check: `UDFRunner`

In [4]:
addone_runner_st = UDFRunner(AddOne())
y_set            = set()
%time addone_runner_st.run(xs, y_set=y_set)

[========================================] 100%

CPU times: user 140 ms, sys: 88 ms, total: 228 ms
Wall time: 1min 40s


In [5]:
y_set == y_set_gold

True

### Next, try parallelism

In [6]:
from multiprocessing import Queue

addone_runner = UDFRunnerMP(AddOne)
y_set         = Queue()
%time addone_runner.run(xs, parallelism=10, y_set=y_set)

CPU times: user 16 ms, sys: 100 ms, total: 116 ms
Wall time: 10.1 s


In [7]:
from Queue import Empty

# Collect from output queue
y_set_out = set()
while True:
    try:
        y_set_out.add(y_set.get(False))
    except Empty:
        break

y_set_out == y_set_gold

True

# Test 2: PubTator Parsing